In [1]:
pwd

'/Users/edwardterrell/Desktop/Training/kindle/strava'

In [2]:
import requests
import json
import time
import os
from pprint import pprint
import pandas as pd
pd.set_option('display.precision', 1)
from clean_convert import cleanup, convert_speed, order_columns

In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))
import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/mlml/bin/python'

In [4]:
from token_manager import get_valid_access_token

access_token = get_valid_access_token()
print("Using access token:", access_token)

# Make an authenticated API call
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)
print(response.json())

Using access token: 8e5e6a3dd85ce84a4003a72c0a7b1b163bb2d747
{'id': 723727, 'username': 'eterrell', 'resource_state': 2, 'firstname': 'Ed', 'lastname': 'Terrell', 'bio': '', 'city': 'Boulder', 'state': 'CO', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2012-07-05T00:04:09Z', 'updated_at': '2025-07-09T12:38:59Z', 'badge_type_id': 1, 'weight': 61.235, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/large.jpg', 'friend': None, 'follower': None}


In [5]:
# Get recent activities
response = requests.get(
    'https://www.strava.com/api/v3/athlete/activities',
    headers={'Authorization': f'Bearer {access_token}'}
)
activities = response.json()
#response.status_code

# Show the first 3 activities with ID and name
for activity in activities[:3]:
    print(f"ID: {activity['id']}, Name: {activity['name']}, Date: {activity['start_date']}")

ID: 15061307314, Name: Nederland exploration and exploitation, Date: 2025-07-09T14:17:28Z
ID: 15048399185, Name: Jamestown and eats at Lucky's Bakery, Date: 2025-07-08T14:44:02Z
ID: 15037283785, Name: Old Stage, Date: 2025-07-07T12:38:38Z


### Build strava_df from activities

In [6]:
# Build out strava_df with only the necessary columns
strava_df = pd.DataFrame(activities)
cols_needed = ['id','name', 'distance', 'moving_time','total_elevation_gain', 'sport_type',
          'average_speed', 'average_heartrate','max_heartrate', 'suffer_score','start_date']
strava_df = strava_df.loc[:,cols_needed]

# drop activities where no heartrate data was collected
strava_df = strava_df.dropna(subset=['average_heartrate'])
#strava_df.head(2)

# process data with function cleanup (from clean_convert.py)
cleanup (strava_df)

# add column: converted_speed ride-mph  and run-min/mile  (from clean_convert.py)
strava_df['converted_speed'] = strava_df.apply(convert_speed, axis=1)


order_columns(strava_df)
strava_df.head(2)

,id,name,distance,moving_time,total_elevation_gain,sport_type,average_speed,average_heartrate,max_heartrate,suffer_score,start_date,date,time,converted_speed
0,15061307314,Nederland exploration and exploitation,33.6,3:13,3260.2,Ride,4.7,109.1,131.0,106.0,2025-07-09 14:17:28+00:00,2025-07-09,194.0,10.4
1,15048399185,Jamestown and eats at Lucky's Bakery,37.5,2:37,2483.6,Ride,6.4,110.9,140.0,101.0,2025-07-08 14:44:02+00:00,2025-07-08,157.8,14.3


### Dataframe approach to create strava_zone_df

In [7]:
# Create a zone dataframe
cols = ['id', 'name', 'date', 'suffer_score']
data = strava_df.iloc[:,[0,1,2,-4]]
strava_zone_df = pd.DataFrame(data, columns=cols)
strava_zone_df.tail(1)

,id,name,date,suffer_score
29,14621663272,The Long Way Home,NaN,NaN


In [8]:
# activity ID to inspect
# activity_id = 15048399185

def get_zones_for_id(activity_id):
    print(activity_id)
    url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
    # Set up Authorization header and make request
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        zones = response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
    # create time_in_zones series
    zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
    time_in_zones = zone_series.apply(lambda z: z['time']/60)
    time_in_zones.index = ['Zone1', 'Zone2', 'Zone3', 'Zone4', 'Zone5']
    time.sleep(.8)
    return (time_in_zones)

In [9]:
zone_df = strava_zone_df['id'].apply(get_zones_for_id)

# Add in summary zones: moderate and intense
zone_df['moderate'] = zone_df[['Zone1', 'Zone2']].sum(axis=1)
zone_df['intense'] = zone_df[['Zone3', 'Zone4', 'Zone5']].sum(axis=1)
# zone_df.head()
# concat both dfs
strava_zone_df = pd.concat([strava_zone_df, zone_df], axis=1)
strava_zone_df

15061307314
15048399185
15037283785
15027982989
15017668877
15006491752
14975645989
14964904782
14954454448
14944687171
14934728422
14925456666
14915319034
14873916201
14860197741
14854414147
14840593381
14767120578
14758807206
14748055178
14737389332
14727760782
14715424784
14705554493
14694429401
14684468262
14672337121
14666228613
14621663272


,id,name,date,suffer_score,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,intense
0,15061307314,Nederland exploration and exploitation,NaN,NaN,3.2e+01,132.7,29.4,0.0,0.0,164.6,29.4
1,15048399185,Jamestown and eats at Lucky's Bakery,NaN,NaN,2.1e+01,109.4,26.1,0.8,0.0,130.8,26.9
2,15037283785,Old Stage,NaN,NaN,2.6e+00,104.8,11.4,0.0,0.0,107.3,11.4
3,15027982989,North dirt roads to Longmont bike paths and Ba...,NaN,NaN,1.6e+01,149.1,1.5,0.0,0.0,165.5,1.5
4,15017668877,Base of LickSkillet++,NaN,NaN,1.8e+01,195.0,8.5,1.4,0.0,212.9,9.9
5,15006491752,Jamestown - Tempo,NaN,NaN,1.1e+00,74.5,72.5,25.0,0.0,75.6,97.4
6,14975645989,Sunshine Canyon and Wallstreet to base of Swit...,NaN,NaN,1.2e+01,178.6,30.6,2.6,0.0,190.4,33.2
7,14964904782,Gravel roads north,NaN,NaN,4.5e+00,145.7,1.8,0.0,0.0,150.2,1.8
8,14954454448,Olde Stage,NaN,NaN,6.0e+00,83.7,4.0,1.3,0.0,89.6,5.3
9,14944687171,Ned gravel loop,NaN,NaN,8.4e+00,51.9,20.7,0.0,0.0,60.3,20.7
